# 参数初始化

对于PyTorch中的各种网络层，我们会需要对参数进行初始化的操作，这里我们以 nn.Linear() 为例子：

```
class Linear(Module):
    def __init__(self, in_features, out_features, bias=True):
    ...
        self.reset_parameters()
    ...
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
```

上述代码摘自 PyTorch [torch.nn.Linear()的源码](http://pytorch.org/docs/master/_modules/torch/nn/modules/linear.html#Linear)

从源代码中可以看到，PyTorch在我们实例化某层的时候，就帮我们自动完成了初始化的操作。但是，有时候，我们可能需要一些不一样的初始化。

比如使用 xavier 方法 来进行初始化，我们该怎么做呢？

在PyTorch中有一个 torch.nn.init 这个包就是用来帮我们完成初始化的。

这个包提供了很多初始化的方法：

* torch.nn.init.constant(tensor, val)
* torch.nn.init.normal(tensor, mean=0, std=1)
* torch.nn.init.xavier_uniform(tensor, gain=1)

详情 http://pytorch.org/docs/master/nn.html#torch-nn-init

In [37]:
import torch
import torch.nn as nn

class LeNet5(nn.Module):
    def __init__(self, in_dim, n_class):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, n_class)
        
        # 参数初始化函数
        for p in self.modules():
            if isinstance(p, nn.Conv2d):
                nn.init.xavier_normal(p.weight.data)
            elif isinstance(p, nn.Linear):
                nn.init.normal(p.weight.data)

    # 向前传播
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
